<a href="https://colab.research.google.com/github/JuanBustamante107517/Lab13-B/blob/main/LAB_13_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
productivity_prediction_of_garment_employees = fetch_ucirepo(id=597)

# data (as pandas dataframes)
X = productivity_prediction_of_garment_employees.data.features
y = productivity_prediction_of_garment_employees.data.targets

# metadata
print(productivity_prediction_of_garment_employees.metadata)

# variable information
print(productivity_prediction_of_garment_employees.variables)

{'uci_id': 597, 'name': 'Productivity Prediction of Garment Employees', 'repository_url': 'https://archive.ics.uci.edu/dataset/597/productivity+prediction+of+garment+employees', 'data_url': 'https://archive.ics.uci.edu/static/public/597/data.csv', 'abstract': 'This dataset includes important attributes of the garment manufacturing process and the productivity of the employees which had been collected manually and also been validated by the industry experts.', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 1197, 'num_features': 14, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['actual_productivity'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2020, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C51S6D', 'creators': [], 'intro_paper': {'ID': 399, 'type': 'NATIVE', 'title': 'Mining the productivity dat

### **(a): Preprocesamiento de datos**

In [ ]:
# Ya deberías tener esto cargado:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

# Cargar dataset
dataset = fetch_ucirepo(id=597)
X = dataset.data.features
y = dataset.data.targets

# Unir en un solo DataFrame
df = pd.concat([X, y], axis=1)
df.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [ ]:
# Ver cantidad de valores faltantes por columna
df.isnull().sum()

,0
date,0
quarter,0
department,0
day,0
team,0
targeted_productivity,0
smv,0
wip,506
over_time,0
incentive,0


In [ ]:
# Opcional: porcentaje de datos faltantes
(df.isnull().sum() / len(df)) * 100

,0
date,0.000000
quarter,0.000000
department,0.000000
day,0.000000
team,0.000000
targeted_productivity,0.000000
smv,0.000000
wip,42.272348
over_time,0.000000
incentive,0.000000


In [ ]:
# Imputar valores faltantes con la mediana (porque es robusta a outliers)
df['wip'].fillna(df['wip'].median(), inplace=True)

<ipython-input-6-524839224>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['wip'].fillna(df['wip'].median(), inplace=True)


In [ ]:
# Detectar y eliminar outliers univariados
def remove_univariate_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

# Aplicamos sobre las variables numéricas
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
df = remove_univariate_outliers(df, numeric_cols)

In [ ]:
from sklearn.ensemble import IsolationForest

# Separar X e y nuevamente
X = df.drop(columns=['actual_productivity'])
y = df['actual_productivity']

# Convertir variables categóricas a numéricas temporalmente (para aplicar Isolation Forest)
X_temp = pd.get_dummies(X, drop_first=True)

# Aplicar Isolation Forest
iso = IsolationForest(contamination=0.01, random_state=42)
outliers = iso.fit_predict(X_temp)

# Eliminar outliers multivariados
mask = outliers != -1
X = X[mask]
y = y[mask]

In [ ]:
# Convertir variables categóricas a dummies (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
X_scaled.shape, y.shape

((653, 78), (653,))

### **(b): Separación de datos y entrenamiento de modelos**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Usamos la mediana como punto de corte para crear dos clases
y_class = (y > y.median()).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42, stratify=y_class)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

print("k-NN:")
for k in [3, 5, 7]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print(f"k = {k} → Accuracy: {accuracy_score(y_test, y_pred):.4f}")

k-NN:
k = 3 → Accuracy: 0.5344
k = 5 → Accuracy: 0.5573
k = 7 → Accuracy: 0.5649


In [ ]:
from sklearn.svm import SVC

print("\nSVM:")
for kernel in ['linear', 'rbf']:
    svm = SVC(kernel=kernel, C=1)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    print(f"Kernel = {kernel} → Accuracy: {accuracy_score(y_test, y_pred):.4f}")


SVM:
Kernel = linear → Accuracy: 0.5649
Kernel = rbf → Accuracy: 0.5649


In [ ]:
from sklearn.linear_model import LogisticRegression

print("\nRegresión Logística:")
for c in [0.1, 1, 10]:
    logreg = LogisticRegression(C=c, max_iter=1000)
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    print(f"C = {c} → Accuracy: {accuracy_score(y_test, y_pred):.4f}")


Regresión Logística:
C = 0.1 → Accuracy: 0.5573
C = 1 → Accuracy: 0.5878
C = 10 → Accuracy: 0.5573


In [ ]:
from sklearn.tree import DecisionTreeClassifier

print("\nÁrbol de Clasificación:")
for depth in [3, 5, 7]:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree.fit(X_train, y_train)
    y_pred = tree.predict(X_test)
    print(f"Max depth = {depth} → Accuracy: {accuracy_score(y_test, y_pred):.4f}")


Árbol de Clasificación:
Max depth = 3 → Accuracy: 0.7023
Max depth = 5 → Accuracy: 0.7634
Max depth = 7 → Accuracy: 0.7863


In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("\nRandom Forest:")
for estimators in [50, 100]:
    rf = RandomForestClassifier(n_estimators=estimators, max_depth=5, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print(f"n_estimators = {estimators} → Accuracy: {accuracy_score(y_test, y_pred):.4f}")


Random Forest:
n_estimators = 50 → Accuracy: 0.7405
n_estimators = 100 → Accuracy: 0.7252


In [ ]:
from sklearn.naive_bayes import GaussianNB

print("\nNaive Bayes:")
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")


Naive Bayes:
Accuracy: 0.4580


In [ ]:
# Por ejemplo, para Random Forest con 100 árboles
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("\nReporte completo de Random Forest (100 árboles):")
print(classification_report(y_test, y_pred))


Reporte completo de Random Forest (100 árboles):
              precision    recall  f1-score   support

           0       0.76      0.67      0.71        66
           1       0.70      0.78      0.74        65

    accuracy                           0.73       131
   macro avg       0.73      0.73      0.72       131
weighted avg       0.73      0.73      0.72       131



### **(c): Optimización de hiperparámetros**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Modelo con parámetros por defecto
rf_default = RandomForestClassifier(random_state=42)
rf_default.fit(X_train, y_train)
y_pred_default = rf_default.predict(X_test)

# Accuracy estándar
acc_default = accuracy_score(y_test, y_pred_default)
print(f"Accuracy (Hiperparámetros estándar): {acc_default:.4f}")

Accuracy (Hiperparámetros estándar): 0.7939


In [ ]:
# Definir rejilla de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring='accuracy'
)

# Entrenar búsqueda en cuadrícula
grid_search.fit(X_train, y_train)

# Mejor modelo
best_grid_model = grid_search.best_estimator_
y_pred_grid = best_grid_model.predict(X_test)
acc_grid = accuracy_score(y_test, y_pred_grid)

print(f"\nAccuracy (Grid Search): {acc_grid:.4f}")
print("Mejores hiperparámetros (Grid Search):")
print(grid_search.best_params_)


Accuracy (Grid Search): 0.7710
Mejores hiperparámetros (Grid Search):
{'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
from scipy.stats import randint

# Definir espacio de búsqueda aleatoria
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=20,  # número de combinaciones aleatorias a probar
    cv=5,
    random_state=42,
    n_jobs=-1,
    scoring='accuracy'
)

# Entrenar búsqueda aleatoria
random_search.fit(X_train, y_train)

# Mejor modelo aleatorio
best_random_model = random_search.best_estimator_
y_pred_random = best_random_model.predict(X_test)
acc_random = accuracy_score(y_test, y_pred_random)

print(f"\nAccuracy (Random Search): {acc_random:.4f}")
print("Mejores hiperparámetros (Random Search):")
print(random_search.best_params_)


Accuracy (Random Search): 0.7634
Mejores hiperparámetros (Random Search):
{'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 67}


In [ ]:
print("\n Comparación final de resultados:")
print(f"1. Estándar:       {acc_default:.4f}")
print(f"2. Grid Search:    {acc_grid:.4f}")
print(f"3. Random Search:  {acc_random:.4f}")

# Mejor modelo
mejor_accuracy = max(acc_default, acc_grid, acc_random)

if mejor_accuracy == acc_default:
    mejor = "Hiperparámetros estándar"
    params = rf_default.get_params()
elif mejor_accuracy == acc_grid:
    mejor = "Grid Search"
    params = grid_search.best_params_
else:
    mejor = "Random Search"
    params = random_search.best_params_

print(f"\n Mejor modelo: {mejor}")
print(f" Accuracy óptimo: {mejor_accuracy:.4f}")
print(f" Hiperparámetros óptimos: {params}")



 Comparación final de resultados:
1. Estándar:       0.7939
2. Grid Search:    0.7710
3. Random Search:  0.7634

 Mejor modelo: Hiperparámetros estándar
 Accuracy óptimo: 0.7939
 Hiperparámetros óptimos: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
